# Hawkes Process  
In this notebook, I attempt to implement the Hawkes Process to learn the distribution of re-rent times.  

This approach failed due to excessive RAM usage, which exceeded the maximum allowed by Google Colab. Even after filtering to include only customer-outfit re-rents that occurred frequently, the issue persisted.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/MyDrive/RecSys_206894495'
#%run /content/drive/MyDrive/RecSys_206894495/models/evaluate_models.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#install
!pip install hawkeslib
!pip install pyarrow

In [ ]:
#import
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from hawkeslib.model.mv_exp import MultivariateExpHawkesProcess as MVHP

In [ ]:
#load data
orders=pd.read_parquet(path+'/archive/data/orders.parquet',engine='pyarrow')

In [ ]:
orders.sort_values('rentalPeriod.start',inplace=True)

# Count the number of orders for each customer-outfit pair
pair_counts = orders.groupby(['customer.id', 'outfit.id']).size()

# Filter out pairs that appear only once
pairs_with_multiple_orders = pair_counts[pair_counts > 7].index

# Filter the orders DataFrame to keep only those pairs
filtered_orders = orders.set_index(['customer.id', 'outfit.id']).loc[pairs_with_multiple_orders].reset_index()
len(filtered_orders)

66

In [ ]:
t = np.array(orders['rentalPeriod.start']).astype('datetime64[D]').astype(int)
c = np.array(orders.index)

mv = MVHP()
mv.fit(t, c)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].stem(mv._mu)
ax[0].set_title("$\mu_k$")
ax[1].imshow(mv._A, cmap="Reds")
ax[1].set_title("$A$")

In [ ]:
"""# verify that the unconditional mean makes sense
mu, alpha, theta = proc.get_params()

N_approx = mu * td[-1] / (1 - alpha)

print(N_approx)
print(len(td))"""

-0.4086188973698293
9


AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# Calculate the rental period duration in days
#orders['rentalPeriod.duration'] = (orders['rentalPeriod.end'] - orders['rentalPeriod.start']).dt.days

In [ ]:
"""# Function to calculate time differences between consecutive orders
def calculate_time_diffs(dates):
    diffs = dates.diff().dropna().dt.days
    return diffs.tolist()

# Create a new DataFrame to store the results
results = []

# Group by customer.id and outfit.id
grouped = orders.groupby(['customer.id', 'outfit.id'])

for (customer_id, outfit_id), group in grouped:
    group = group.sort_values('rentalPeriod.start')
    first_order_date = group['rentalPeriod.start'].iloc[0]
    time_diffs = calculate_time_diffs(group['rentalPeriod.start'])

    result = {
        'customer.id': customer_id,
        'outfit.id': outfit_id,
        'first_order_date': first_order_date
    }

    for i, diff in enumerate(time_diffs):
        result[f'time_between_order_{i+1}_and_{i+2}'] = diff

    results.append(result)

# Create the final DataFrame
final_df = pd.DataFrame(results)
final_df
"""